### Week of 9/13 - 9/20 + Week of 9/27 - 10/4 + wherever we're at now

pip install:
scipy
seaborn
pandas
tables

Relevant function from comb-building, addition/modification for foraging below

In [5]:
import pandas as pd
import numpy as np
import pickle, gzip
import matplotlib.pyplot as plt
%matplotlib inline
#from toolbox import defaultFigure
#defaultFigure.setDefaultParams()
#from toolbox import gifsicle
from importlib import reload  
import datetime as dt
import random
import cProfile
#from tqdm import tqdm

In [23]:
trajectory_days_to_load = [4,]

In [24]:
trajectory_data_dir = '../bee-data/beetrajectories_days_000_to_049/'

In [26]:
#dfTrajList = [ pd.read_hdf('{}beetrajectories_{:03d}.hdf'.format(trajectory_data_dir,daynum)) for daynum in days_to_load ]
#dfTraj = pd.concat(dfTrajList) # avoid explicitly creating list to save memory
filenames = [ '{}beetrajectories_{:03d}.hdf'.format(trajectory_data_dir,daynum) for daynum in trajectory_days_to_load ]
dfTraj = pd.concat(map(pd.read_hdf,filenames))

In [27]:
def interaction_events(dfTraj,beeID,minTime,maxTime,distance_thresh=10):
    """
    Returns dataFrame listing interaction events during which the bee with ID beeID is nearby another bee, within
    the time window between minTime and maxTime.  Here "nearby" means within a distance threshold and on the same 
    side of the hive.

    dfTraj                   : Trajectory dataframe containing framenumbers, (x,y) coordinates, and camera information.
    minTime                  : Minimum time to scan for interaction events, as a datetime object
    maxTime                  : Maximum time to scan for interaction events, as a datetime object
    distance_thresh (10)     : Distance in pixels within which two bees must be imaged in the same frame to count as an interaction
    """
    # select trajectory data from all bees from the desired time window
    minFrame = datetime_to_framenum(minTime)
    maxFrame = datetime_to_framenum(maxTime)
    dfTraj_selected = dfTraj[(minFrame < dfTraj['framenum']) & (dfTraj['framenum'] < maxFrame)]

    # select trajectory data from the selected bee
    dfTraj_selected_bee = dfTraj_selected[dfTraj_selected['uid']==beeID]
    
    # loop over time looking for interaction events
    interaction_data = []
    for frame in dfTraj_selected_bee['framenum'].unique():
        all_bee_frame_data = dfTraj_selected[dfTraj_selected['framenum']==frame]
        
        # get selected bee location for this time frame
        selected_bee_frame_data = all_bee_frame_data[all_bee_frame_data['uid']==beeID]
        beex = selected_bee_frame_data['x'].iloc[0],
        beey = selected_bee_frame_data['y'].iloc[0]
        beecamera = selected_bee_frame_data['camera'].iloc[0]

        # find distances from all other bees seen by the same camera (same side of hive) at this time frame
        all_bee_frame_data_same_side = all_bee_frame_data[all_bee_frame_data['camera']==beecamera]
        distances = pd.Series(np.linalg.norm([all_bee_frame_data_same_side['x']-beex,
                                              all_bee_frame_data_same_side['y']-beey],axis=0),
                              index=all_bee_frame_data_same_side['uid'])

        # append interaction data
        for other_bee in distances[distances < distance_thresh].index:
            if other_bee != beeID:
                other_bee_frame_data = all_bee_frame_data_same_side[all_bee_frame_data_same_side['uid']==other_bee]
                assert(len(other_bee_frame_data)==1) # should only have one timepoint for one bee in one frame
                interaction_data.append([frame,
                                         other_bee,
                                         other_bee_frame_data['x'].iloc[0],
                                         other_bee_frame_data['y'].iloc[0],
                                         other_bee_frame_data['camera'].iloc[0],
                                         distances[other_bee]])
    
    return pd.DataFrame(interaction_data,columns=['framenum','uid','x','y','camera','distance (pixels)'])

In [29]:
def number_of_interactions(dfTraj, beeID, minTime, maxTime, distance_thresh):
    interaction_matrix = interaction_events(dfTraj, beeID, minTime, maxTime, distance_thresh)
    return length(interaction_matrix)
    

In [30]:
def interactions_ages(dfTraj, beeID, minTime, maxTime, distance_thresh):
    interaction_matrix = interaction_events(dfTraj, beeID, minTime, maxTime, distance_thresh)
    ages_list = []
    for row in interaction_matrix:
        ages_list.append('''uid.age''')
    return ages_list

ages_list = interactions_ages(dfTraj, beeID, minTime, maxTime, distance_thresh)
average_interaction_age = 0
for age in ages_list:
    average_interaction_age += age
average_interaction_age /= length(ages_list)

NameError: name 'beeID' is not defined

In [4]:
def time_to_forage_list(dfTraj, beeID, minTime, maxTime, distance_thresh):
    interaction_matrix = interaction_events(dfTraj, beeID, minTime, maxTime, distance_thresh)
    forage_list = []
    for row in interaction_matrix:
        forage_list.append('''time_to_forage(uid)''')
    return forage_list

foraging_list = time_to_forage_list(dfTraj, beeID, minTime, maxTime, distance_thresh)
average_forage_time = 0
for time in foraging_list:
    average_forage_time += time
average_forage_time /= length(foraging_list)

NameError: name 'dfTraj' is not defined

In [34]:
# a plot to show an individual bee's interactions against the time they occur and to display total interactions over a time frame
for beeID in [672,590,665,582]: #placeholders
    interaction_num = number_of_interactions(dfTraj, beeID, minTime, maxTime, distance_thresh)
    times = [startframe, endframe]
    plt.plot(times[::(endframe - startframe)/10],interaction_num,label=beeID)
plt.legend()
plt.xlabel('Time (frames)' + str(startframe) + ', ' str(endframe))
plt.ylabel('interactions per unit time' + 'total interactions =' + interaction_num);

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2113258322.py, line 7)